In [1]:
import pandas as pd
from tqdm import tqdm as tqdm
from binance.client import Client

In [2]:
client = Client() # Initialize the Client from binance

In [3]:
info = client.get_exchange_info() # Here I obtain all data from binance exchange
symbols = [x['symbol'] for x in info['symbols']] # here I filter only "symbols" (using List comprehension)

In [4]:
exclude = ['UP', 'DOWN', 'BEAR', 'BULL'] # Here is to exclude all leveraged tokens
non_lev = [symbol for symbol in symbols if all (excludes not in symbol for excludes in exclude)] # (using List comprehension)
relevant = [symbol for symbol in non_lev if symbol.endswith('USDT')] # Here is to filter all pairs that ends with "USDT" (using List comprehension)

In [5]:
klines = {} # Empty dictionary where it will fill with the kline data
for symbol in tqdm(relevant): # a for loop, looping throught the "relevant" data using "tqdm" to get a progress bar
    klines[symbol] = client.get_historical_klines(symbol, '1m', '1 hour ago UTC') # creating the keys for all symbols using the info from binance. I provide the symbol, the interval and how much time i wanna go back

100%|██████████| 337/337 [03:15<00:00,  1.73it/s]


In [13]:
returns, symbols = [], [] # Get returns for all symbols appending thats info into both dictionaries

for symbol in relevant: # a loop through al the "relevant" symbols with USDT
    
    if len(klines[symbol]) > 0: # Going to check and exclude symbols with no entries ("returns")
        
        cumreturn = (pd.DataFrame(klines[symbol])[4].astype(float).pct_change() +1).prod() -1 # Here I define the cumulative return creating a data frame for each iteration, indexing for the 4th column, converting to a float value
        returns.append(cumreturn) # append the "cumreturn" values to "returns" dictionary
        symbols.append(symbol) # append the "symbol" values (symbols) to the symbols dictionary

In [14]:
returndf = pd.DataFrame(returns, index=symbols, columns=['ret'])

In [16]:
print(returndf)

               ret
BTCUSDT   0.000494
ETHUSDT  -0.001175
BNBUSDT  -0.007639
NEOUSDT  -0.000446
LTCUSDT  -0.002308
...            ...
BTTCUSDT  0.004525
ACAUSDT   0.001884
ANCUSDT  -0.001351
XNOUSDT   0.023820
WOOUSDT  -0.006567

[317 rows x 1 columns]


In [17]:
print(returndf.ret.nlargest(10))

FIOUSDT     0.042254
QIUSDT      0.034346
XNOUSDT     0.023820
MIRUSDT     0.023372
LTOUSDT     0.018090
AGLDUSDT    0.018088
RNDRUSDT    0.016553
UMAUSDT     0.016150
KNCUSDT     0.016121
FTMUSDT     0.015764
Name: ret, dtype: float64
